In [79]:
import pandas as pd
import os
from torch.utils.data import Dataset, DataLoader
from torchvision.io import read_image
from torchvision.models.detection.faster_rcnn import FastRCNNPredictor
import torchvision
import ast
import torch

KAGGLE_PATH_ANNOTATIONS = '/kaggle/input/tensorflow-great-barrier-reef/train.csv'
KAGGLE_PATH_IMG_DIR = '/kaggle/input/tensorflow-great-barrier-reef/train_images/'
LOCAL_PATH_ANNOTATIONS = 'data/train.csv'
LOCAL_PATH_IMG_DIR = 'data/train_images/'


# TODO: pewnie można zrobić zmienne globalne z directory path, żeby podmieniać na kagglową jak puszczamy w kagglu i na własną, jak puszczamy lokalnie
# *ewentualnie lokalnie ustawić jak w kaggle xd

In [81]:
class StarfishDataset(Dataset):
    def __init__(self,
                 annotations_file=LOCAL_PATH_ANNOTATIONS,
                 img_dir=LOCAL_PATH_IMG_DIR
                 ):
        self.img_labels = pd.read_csv(annotations_file)
        self.annotated = self.img_labels[self.img_labels['annotations'] != '[]']  # get only annotated frames
        self.img_dir = img_dir

    def __len__(self):
        return len(self.annotated)

    def __getitem__(self, idx):
        image = read_image(os.path.join(self.img_dir, 'video_{}'.format(self.annotated.iloc[idx][0]),
                                        '{}.jpg'.format(self.annotated.iloc[idx][2])))
        label = self.annotated.iloc[idx][-1]
        parsed_label = ast.literal_eval(label[
                                        1:-1])  # a label is initially a string representing a dictionary, now it is changed to dictionary object, not string
        return image, parsed_label


In [82]:
def get_model(num_classes):
    # load an object detection model pre-trained on COCO
    model = torchvision.models.detection.fasterrcnn_resnet50_fpn(
        pretrained=True)  # get the number of input features for the classifier
    in_features = model.roi_heads.box_predictor.cls_score.in_features
    # replace the pre-trained head with a new on
    model.roi_heads.box_predictor = FastRCNNPredictor(in_features, num_classes)

    return model

In [86]:
torch.manual_seed(1)
dataset = StarfishDataset()
train_size = int(0.8 * len(dataset))
test_size = len(dataset) - train_size
train_dataset, test_dataset = torch.utils.data.random_split(dataset, [train_size, test_size])
print('Train dataset: {} instances, test dataset: {}'.format(len(train_dataset), len(test_dataset)))

data_loader = torch.utils.data.DataLoader(
    train_dataset, batch_size=2, shuffle=False, num_workers=4)
data_loader_test = torch.utils.data.DataLoader(
    test_dataset, batch_size=1, shuffle=False, num_workers=4)

device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')

num_classes = 2 # starfish and not starfish I guess
model = get_model(num_classes)
model.to(device)
params = [p for p in model.parameters() if p.requires_grad]
optimizer = torch.optim.SGD(params, lr=0.005, momentum=0.9, weight_decay=0.0005)
lr_scheduler = torch.optim.lr_scheduler.StepLR(optimizer,step_size=3, gamma=0.1)

Train dataset: 3935 instances, test dataset: 984


Downloading: "https://download.pytorch.org/models/fasterrcnn_resnet50_fpn_coco-258fb6c6.pth" to /home/witold/.cache/torch/hub/checkpoints/fasterrcnn_resnet50_fpn_coco-258fb6c6.pth


  0%|          | 0.00/160M [00:00<?, ?B/s]

In [ ]:
# TODO: training loop